In [1]:
#User inputs for date. Hopefully this will be the only manual component
month = 1
day = 13
year = 2023

In [2]:
#In an attempt to streamline this code, I will combine preexisting scripts into one notebook
#import the necessary python packages
import sys

selenium_path = r'C:\Users\Kimbe\anaconda3\Lib\site-packages\selenium'
PYSOLR_PATH = r'c:\users\kimbe\appdata\local\programs\python\python312\lib\site-packages'

if not PYSOLR_PATH in sys.path:
    sys.path.append(PYSOLR_PATH)

if not selenium_path in sys.path:
    sys.path.append(selenium_path)

import pandas
import numpy
import requests
import bs4
import re
import undetected_chromedriver as uc
import selenium
import os
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import time
import datetime
import re
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def get_case_list(url):
    #Declare a list to store the results
    case_data_text = []
    
    #Launch the headless browser
    driver = uc.Chrome()

    #Navigate to the CaseNet URL
    #driver.get(url)
    try:
        driver.get(url)
        driver.maximize_window()
        driver.execute_script("document.body.style.zoom='60%'")
        
    except Exception as e:
        print(e.message)

    #Give the page ample time to load
    time.sleep(10)
    #wait = WebDriverWait(driver, 10)

    #Scroll to ensure the "Next" button is visible
   # element = driver.find_element(By.ID, "searchResult_next")
   # actions = ActionChains(driver)
   # actions.move_to_element(element).perform()
    
    #Wait for the page to load
    time.sleep(2) 
    i = 0
    success = 0
    sequential_errors = 0
    
    #Loop through all pages of the table. Find all table rows, and extract all table data, saving the data in a list
    while True:
        try:
            time.sleep(1)
            WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.ID, "searchResult_next"))).click()
            html_content = driver.page_source
            soup = bs4.BeautifulSoup(html_content, 'html.parser')

            #Get all table rows on the page
            trs = soup.find_all(attrs={"role": "row"}) 

            for tr in trs:
                tds = tr.find_all('td') # output data by referring to the list items by position
                case_data_text.append(tds)
                print("Success: " + str(success))
                success = success+1
                sequential_errors = 0
                #print(f'Next------------------------------------') # for clarity

        except TimeoutException:
            break
        else:
            sequential_errors = sequential_errors + 1
            print("error " + str(i))
            print(sequential_errors)
            if sequential_errors > 10:
                driver.close()
                return case_data_text
            else:
                numpy.nan
                
            if i >= 1000:
                driver.close()
                return case_data_text
            else:
                 i = i + 1   
            
    driver.close()
    return case_data_text

In [4]:
#Define function to parse a given row in the case list data
def parse_case_text(text): 
    try:
        soup = bs4.BeautifulSoup(text)
        date_filed = str(soup.find_all('td')[1]).replace('<td>', '').replace('</td>', '')
        case_number = str(str(soup.find_all('td')[2]).replace('<td>', '').replace('</td>', '').split('"', 4)[4]).replace(">", '').replace("</a", '')
        style_of_case = str(soup.find_all('td')[3]).replace('<td>', '').replace('</td>', '')
        case_type =  str(soup.find_all('td')[4]).replace('<td>', '').replace('</td>', '')
        location =  str(soup.find_all('td')[5]).replace('<td>', '').replace('</td>', '')
        link_text = 'https://www.courts.mo.gov' + str(soup.a).replace('<a class="stretched-link text-justify" href="', '').split('"', 1)[0] + '&inputVO.courtId=CT21'

        array = pandas.DataFrame({"date_filed": [date_filed], 'case_number': [case_number], 'style_of_case': [style_of_case], 'case_type': [case_type], 'location': [location], 'link_text': [link_text]})
        return array
    except:
        print("Skipping row: " + str(text))
        array = pandas.DataFrame({"date_filed": ['Skipped'], 'case_number': ['Skipped'], 'style_of_case': ['Skipped'], 'case_type': ['Skipped'], 'location': ['Skipped'], 'link_text': ['Skipped']})
        return 


In [5]:
#Define a function to parse the case header HTML text
def parse_case_header(text, case_number):
    soup = bs4.BeautifulSoup(text, 'html.parser')
    
    try:
        objects = str(soup.find_all(attrs={"class": 'col-lg-5 col-md-6 col-sm-6 col-xs-12 odd'})).replace('<strong>', '').replace('<div class="col-lg-5 col-md-6 col-sm-6 col-xs-12 odd', '').replace('">', '').replace('</strong>', '').replace('</div>', '').replace('\n', '').replace("[", '').replace(']', '')
        values = str(soup.find_all(attrs={"class": 'col-lg-7 col-md-6 col-sm-6 col-xs-12 even'})).replace('<div class="col-lg-7 col-md-6 col-sm-6 col-xs-12 even', '').replace('">', '').replace('</div>', '').replace('\n', '').replace("[", '').replace(']', '')
 
        column_names = [x.strip() for x in objects.split(',')]

        case_data = [x.strip() for x in values.split(',')]

        #Clean up surname issues
        if 'Judge/Commissioner Assigned' in column_names:
            case_data[0] = case_data[0] + ', ' + case_data[1]
            case_data.pop(1)

        if 'Judge/Commissioner At Disposition' in column_names:
            first_name_index = len(case_data) - 1
            last_name_index = first_name_index - 1
            case_data[last_name_index] = case_data[last_name_index] + ', ' + case_data[first_name_index]
            case_data.pop(first_name_index)

        if 'Financial Information' in column_names:
            fin_info_index = len(case_data)
            column_names.pop(fin_info_index)

        case_details = pandas.DataFrame([case_data], columns = [column_names])
        
        #When a case doesn't have a field for Judge/Commissioner At Disposition, add a 
        #column containing NaN
        if 'Judge/Commissioner At Disposition' in case_details.columns:
            numpy.nan
        else:
            case_details['Judge/Commissioner At Disposition'] = numpy.nan
            
        #Create a column for case number
        case_details['case_number'] = case_number
        return case_details
    except:
        print("ERROR on case_number " + case_number)
        return

In [6]:
#Create a date variable for the query, and create an empty array to store webscraper results
start_date = str(month) + '%2F' + str(day) + '%2F' + str(year)
start_date_string = str(datetime.datetime(year, month, day).strftime('%Y-%m-%d'))

#URL to query cases in St. Louis County (aka the 21st Circuit Court)
casenet_url = 'https://www.courts.mo.gov/cnet/searchResult.do?countyCode=SLC&newSearch=Y&courtCode=CT21&startDate=' + start_date + '&caseStatus=A&caseType=All&locationCode='

In [7]:
casenet_url

'https://www.courts.mo.gov/cnet/searchResult.do?countyCode=SLC&newSearch=Y&courtCode=CT21&startDate=1%2F13%2F2023&caseStatus=A&caseType=All&locationCode='

In [8]:
#Query all cases occurring in the seven day period starting with the user-defined date.
cases_text = get_case_list(casenet_url)
#I set th error max to 1200 when it actually needs to be 500-ish

Success: 0
Success: 1
Success: 2
Success: 3
Success: 4
Success: 5
Success: 6
Success: 7
Success: 8
Success: 9
Success: 10
error 0
1
Success: 11
Success: 12
Success: 13
Success: 14
Success: 15
Success: 16
Success: 17
Success: 18
Success: 19
Success: 20
Success: 21
error 1
1
Success: 22
Success: 23
Success: 24
Success: 25
Success: 26
Success: 27
Success: 28
Success: 29
Success: 30
Success: 31
Success: 32
error 2
1
Success: 33
Success: 34
Success: 35
Success: 36
Success: 37
Success: 38
Success: 39
Success: 40
Success: 41
Success: 42
Success: 43
error 3
1
Success: 44
Success: 45
Success: 46
Success: 47
Success: 48
Success: 49
Success: 50
Success: 51
Success: 52
Success: 53
Success: 54
error 4
1
Success: 55
Success: 56
Success: 57
Success: 58
Success: 59
Success: 60
Success: 61
Success: 62
Success: 63
Success: 64
Success: 65
error 5
1
Success: 66
Success: 67
Success: 68
Success: 69
Success: 70
Success: 71
Success: 72
Success: 73
Success: 74
Success: 75
Success: 76
error 6
1
Success: 77
Succ

Success: 594
Success: 595
Success: 596
Success: 597
Success: 598
Success: 599
Success: 600
Success: 601
Success: 602
Success: 603
Success: 604
error 54
1
Success: 605
Success: 606
Success: 607
Success: 608
Success: 609
Success: 610
Success: 611
Success: 612
Success: 613
Success: 614
Success: 615
error 55
1
Success: 616
Success: 617
Success: 618
Success: 619
Success: 620
Success: 621
Success: 622
Success: 623
Success: 624
Success: 625
Success: 626
error 56
1
Success: 627
Success: 628
Success: 629
Success: 630
Success: 631
Success: 632
Success: 633
Success: 634
Success: 635
Success: 636
Success: 637
error 57
1
Success: 638
Success: 639
Success: 640
Success: 641
Success: 642
Success: 643
Success: 644
Success: 645
Success: 646
Success: 647
Success: 648
error 58
1
Success: 649
Success: 650
Success: 651
Success: 652
Success: 653
Success: 654
Success: 655
Success: 656
Success: 657
Success: 658
Success: 659
error 59
1
Success: 660
Success: 661
Success: 662
Success: 663
Success: 664
Success: 66

Success: 1177
Success: 1178
Success: 1179
Success: 1180
Success: 1181
Success: 1182
Success: 1183
Success: 1184
Success: 1185
Success: 1186
Success: 1187
error 107
1
Success: 1188
Success: 1189
Success: 1190
Success: 1191
Success: 1192
Success: 1193
Success: 1194
Success: 1195
Success: 1196
Success: 1197
Success: 1198
error 108
1
Success: 1199
Success: 1200
Success: 1201
Success: 1202
Success: 1203
Success: 1204
Success: 1205
Success: 1206
Success: 1207
Success: 1208
Success: 1209
error 109
1
Success: 1210
Success: 1211
Success: 1212
Success: 1213
Success: 1214
Success: 1215
Success: 1216
Success: 1217
Success: 1218
Success: 1219
Success: 1220
error 110
1
Success: 1221
Success: 1222
Success: 1223
Success: 1224
Success: 1225
Success: 1226
Success: 1227
Success: 1228
Success: 1229
Success: 1230
Success: 1231
error 111
1
Success: 1232
Success: 1233
Success: 1234
Success: 1235
Success: 1236
Success: 1237
Success: 1238
Success: 1239
Success: 1240
Success: 1241
Success: 1242
error 112
1
Succ

Success: 1727
Success: 1728
Success: 1729
Success: 1730
Success: 1731
Success: 1732
Success: 1733
Success: 1734
Success: 1735
Success: 1736
Success: 1737
error 157
1
Success: 1738
Success: 1739
Success: 1740
Success: 1741
Success: 1742
Success: 1743
Success: 1744
Success: 1745
Success: 1746
Success: 1747
Success: 1748
error 158
1
Success: 1749
Success: 1750
Success: 1751
Success: 1752
Success: 1753
Success: 1754
Success: 1755
Success: 1756
Success: 1757
Success: 1758
Success: 1759
error 159
1
Success: 1760
Success: 1761
Success: 1762
Success: 1763
Success: 1764
Success: 1765
Success: 1766
Success: 1767
Success: 1768
Success: 1769
Success: 1770
error 160
1
Success: 1771
Success: 1772
Success: 1773
Success: 1774
Success: 1775
Success: 1776
Success: 1777
Success: 1778
Success: 1779
Success: 1780
Success: 1781
error 161
1
Success: 1782
Success: 1783
Success: 1784
Success: 1785
Success: 1786
Success: 1787
Success: 1788
Success: 1789
Success: 1790
Success: 1791
Success: 1792
error 162
1
Succ

Success: 2277
Success: 2278
Success: 2279
Success: 2280
Success: 2281
Success: 2282
Success: 2283
Success: 2284
Success: 2285
Success: 2286
Success: 2287
error 207
1
Success: 2288
Success: 2289
Success: 2290
Success: 2291
Success: 2292
Success: 2293
Success: 2294
Success: 2295
Success: 2296
Success: 2297
Success: 2298
error 208
1
Success: 2299
Success: 2300
Success: 2301
Success: 2302
Success: 2303
Success: 2304
Success: 2305
Success: 2306
Success: 2307
Success: 2308
Success: 2309
error 209
1
Success: 2310
Success: 2311
Success: 2312
Success: 2313
Success: 2314
Success: 2315
Success: 2316
Success: 2317
Success: 2318
Success: 2319
Success: 2320
error 210
1
Success: 2321
Success: 2322
Success: 2323
Success: 2324
Success: 2325
Success: 2326
Success: 2327
Success: 2328
Success: 2329
Success: 2330
Success: 2331
error 211
1
Success: 2332
Success: 2333
Success: 2334
Success: 2335
Success: 2336
Success: 2337
Success: 2338
Success: 2339
Success: 2340
Success: 2341
Success: 2342
error 212
1
Succ

...

Success: 6127
Success: 6128
Success: 6129
Success: 6130
Success: 6131
Success: 6132
Success: 6133
Success: 6134
Success: 6135
Success: 6136
Success: 6137
error 557
1
Success: 6138
Success: 6139
Success: 6140
Success: 6141
Success: 6142
Success: 6143
Success: 6144
Success: 6145
Success: 6146
Success: 6147
Success: 6148
error 558
1
Success: 6149
Success: 6150
Success: 6151
Success: 6152
Success: 6153
Success: 6154
Success: 6155
Success: 6156
Success: 6157
Success: 6158
Success: 6159
error 559
1
Success: 6160
Success: 6161
Success: 6162
Success: 6163
Success: 6164
Success: 6165
Success: 6166
Success: 6167
Success: 6168
Success: 6169
Success: 6170
error 560
1
Success: 6171
Success: 6172
Success: 6173
Success: 6174
Success: 6175
Success: 6176
Success: 6177
Success: 6178
Success: 6179
Success: 6180
Success: 6181
error 561
1
Success: 6182
Success: 6183
Success: 6184
Success: 6185
Success: 6186
Success: 6187
Success: 6188
Success: 6189
Success: 6190
Success: 6191
Success: 6192
error 562
1
Succ

Success: 6640
Success: 6641
Success: 6642
Success: 6643
Success: 6644
error 644
1
Success: 6645
Success: 6646
Success: 6647
Success: 6648
Success: 6649
error 645
1
Success: 6650
Success: 6651
Success: 6652
Success: 6653
Success: 6654
error 646
1
Success: 6655
Success: 6656
Success: 6657
Success: 6658
Success: 6659
error 647
1
Success: 6660
Success: 6661
Success: 6662
Success: 6663
Success: 6664
error 648
1
Success: 6665
Success: 6666
Success: 6667
Success: 6668
Success: 6669
error 649
1
Success: 6670
Success: 6671
Success: 6672
Success: 6673
Success: 6674
error 650
1
Success: 6675
Success: 6676
Success: 6677
Success: 6678
Success: 6679
error 651
1
Success: 6680
Success: 6681
Success: 6682
Success: 6683
Success: 6684
error 652
1
Success: 6685
Success: 6686
Success: 6687
Success: 6688
Success: 6689
error 653
1
Success: 6690
Success: 6691
Success: 6692
Success: 6693
Success: 6694
error 654
1
Success: 6695
Success: 6696
Success: 6697
Success: 6698
Success: 6699
error 655
1
Success: 6700
Su

Success: 7140
Success: 7141
Success: 7142
Success: 7143
Success: 7144
error 744
1
Success: 7145
Success: 7146
Success: 7147
Success: 7148
Success: 7149
error 745
1
Success: 7150
Success: 7151
Success: 7152
Success: 7153
Success: 7154
error 746
1
Success: 7155
Success: 7156
Success: 7157
Success: 7158
Success: 7159
error 747
1
Success: 7160
Success: 7161
Success: 7162
Success: 7163
Success: 7164
error 748
1
Success: 7165
Success: 7166
Success: 7167
Success: 7168
Success: 7169
error 749
1
Success: 7170
Success: 7171
Success: 7172
Success: 7173
Success: 7174
error 750
1
Success: 7175
Success: 7176
Success: 7177
Success: 7178
Success: 7179
error 751
1
Success: 7180
Success: 7181
Success: 7182
Success: 7183
Success: 7184
error 752
1
Success: 7185
Success: 7186
Success: 7187
Success: 7188
Success: 7189
error 753
1
Success: 7190
Success: 7191
Success: 7192
Success: 7193
Success: 7194
error 754
1
Success: 7195
Success: 7196
Success: 7197
Success: 7198
Success: 7199
error 755
1
Success: 7200
Su

Success: 7640
Success: 7641
Success: 7642
Success: 7643
Success: 7644
error 844
1
Success: 7645
Success: 7646
Success: 7647
Success: 7648
Success: 7649
error 845
1
Success: 7650
Success: 7651
Success: 7652
Success: 7653
Success: 7654
error 846
1
Success: 7655
Success: 7656
Success: 7657
Success: 7658
Success: 7659
error 847
1
Success: 7660
Success: 7661
Success: 7662
Success: 7663
Success: 7664
error 848
1
Success: 7665
Success: 7666
Success: 7667
Success: 7668
Success: 7669
error 849
1
Success: 7670
Success: 7671
Success: 7672
Success: 7673
Success: 7674
error 850
1
Success: 7675
Success: 7676
Success: 7677
Success: 7678
Success: 7679
error 851
1
Success: 7680
Success: 7681
Success: 7682
Success: 7683
Success: 7684
error 852
1
Success: 7685
Success: 7686
Success: 7687
Success: 7688
Success: 7689
error 853
1
Success: 7690
Success: 7691
Success: 7692
Success: 7693
Success: 7694
error 854
1
Success: 7695
Success: 7696
Success: 7697
Success: 7698
Success: 7699
error 855
1
Success: 7700
Su

Success: 8140
Success: 8141
Success: 8142
Success: 8143
Success: 8144
error 944
1
Success: 8145
Success: 8146
Success: 8147
Success: 8148
Success: 8149
error 945
1
Success: 8150
Success: 8151
Success: 8152
Success: 8153
Success: 8154
error 946
1
Success: 8155
Success: 8156
Success: 8157
Success: 8158
Success: 8159
error 947
1
Success: 8160
Success: 8161
Success: 8162
Success: 8163
Success: 8164
error 948
1
Success: 8165
Success: 8166
Success: 8167
Success: 8168
Success: 8169
error 949
1
Success: 8170
Success: 8171
Success: 8172
Success: 8173
Success: 8174
error 950
1
Success: 8175
Success: 8176
Success: 8177
Success: 8178
Success: 8179
error 951
1
Success: 8180
Success: 8181
Success: 8182
Success: 8183
Success: 8184
error 952
1
Success: 8185
Success: 8186
Success: 8187
Success: 8188
Success: 8189
error 953
1
Success: 8190
Success: 8191
Success: 8192
Success: 8193
Success: 8194
error 954
1
Success: 8195
Success: 8196
Success: 8197
Success: 8198
Success: 8199
error 955
1
Success: 8200
Su

In [9]:
#Convert the case list to a dataframe and save it as a csv 
cases_df = pandas.DataFrame({'CASES_HTML':cases_text})
cases_df.to_csv("data\\case_data\\" + start_date_string+ "_cases.csv", index = False)

In [10]:
#cases_df = pandas.read_csv("data\\case_data\\2023-01-07_cases.csv")
#I'm getting 4,762 distinct cases. This matches the true number of CaseNet search results. Perfecto!
cases_df = cases_df['CASES_HTML'].drop_duplicates().reset_index()


In [11]:
max(cases_df.index)

5704

In [12]:
#From the case list HTML data, extract details about the case
case_details_df = pandas.DataFrame()
for i in range(0, max(cases_df.index)):
    text = str(cases_df.iloc[i, 1])
    df = parse_case_text(text)
    
    if "case_details_df" in locals():
        case_details_df = case_details_df.append(df)
    else:
        case_details_df = df


Skipping row: [<td aria-controls="searchResult" aria-label="&amp;nbsp;: activate to sort column descending" aria-sort="ascending" class="sorting_asc" colspan="1" rowspan="1" style="width: 44px;" tabindex="0"> </td>]


In [13]:
#Save case details data in a csv
case_details_df.to_csv("data\\case_details_data\\"+start_date_string+"_case_details.csv", index = False)

In [14]:
#One case was the search result header, which got subtracted from the total. 
#Since the true number of cases is my data is missing one case.
case_details_df['case_number'].drop_duplicates().size

5703

In [15]:
#Navigate to each case's webpage and scrape any relevant details
dates = case_details_df['date_filed'].drop_duplicates().reset_index()

error_cases = []


for i in range(0, max(dates.index)):
    driver = uc.Chrome()

    #Get all cases for a particular day
    cases_subset = case_details_df.loc[case_details_df['date_filed'] == dates.iloc[i,1]]
    cases_subset = cases_subset.reset_index()
    date_name = str(datetime.datetime.strptime(dates.iloc[i,1], '%m/%d/%Y').strftime('%Y-%m-%d'))
    
    #Get the highest index value
    stop = max(cases_subset.index)
    
    #Loop through the list of cases and scrape the header data. Save the data in a pandas dataframe.
    for j in range(0, stop):
            try:
                driver.get(cases_subset.iloc[j, 6])
                case_number = cases_subset.iloc[j, 2]
                                
                #Give the page time to load
                time.sleep(1)
                html_data = driver.page_source
                output = parse_case_header(html_data, case_number)

                if "case_headers" in locals():
                    case_headers = case_headers.append(output)
                else:
                    case_headers = output
            except:
                print('error on case number ' + str(case_number))
                error_cases.append(str(case_number))
    
    file_name = 'data\\header_data\\' + date_name + '_case_header_details.csv'
    case_headers.to_csv(file_name, index = False)
    del(case_headers)
    driver.close()


ERROR on case_number 23SL-CC00346
ERROR on case_number 23SL-CC00342


In [16]:
f1 = pandas.read_csv('data\\header_data\\2023-01-13_case_header_details.csv')
f2 = pandas.read_csv('data\\header_data\\2023-01-14_case_header_details.csv')
f3 = pandas.read_csv('data\\header_data\\2023-01-15_case_header_details.csv')
f4 = pandas.read_csv('data\\header_data\\2023-01-16_case_header_details.csv')
f5 = pandas.read_csv('data\\header_data\\2023-01-17_case_header_details.csv')
f6 = pandas.read_csv('data\\header_data\\2023-01-18_case_header_details.csv')

In [17]:
#I'm missing roughly 600 cases. Do I need to slow down the loop so it has 2 seconds to load?
f1['case_number'].size + f2['case_number'].size + f3['case_number'].size + f4['case_number'].size + f5['case_number'].size + f6['case_number'].size

4031